## This lab demonstrates web scrapting and then adding latitude & longitude information for each geographic location

#### Import necessary Libraries

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install beautifulsoup4
from bs4 import BeautifulSoup

#### Scrap wikipedia page and read html to convert wikipedia table into pandas dataframe

In [4]:
wikiurl=("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')
wikitable=soup.find('table',{'class':"wikitable sortable"})

In [6]:
# Read html and create a list
df=pd.read_html(str(wikitable))
#convert list into dataframe
df=pd.DataFrame(df[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Ignore the records where Borough is 'Not assigned'

In [7]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Combine multiple Neighbourhoods with same Postal code

In [8]:
# Combine using comma. Although, this is already taken care in the latest wiki page table , so this code is not required

df['Neighbourhood'] = df.groupby(['Postal Code'])['Neighbourhood'].transform(lambda x : ','.join(x)) 
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Update value of Neighbourhood same as corresponding Borough in case where Neighbour = 'Not assigned'

In [9]:
df.loc[(df.Neighbourhood == 'Not assigned'),'Neighbourhood'] = df['Borough']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Check the size of the dataframe

In [10]:
df.shape

(103, 3)

#### Display Latitide & Longitude for each Postal code using csv file

In [11]:
# dataframe to contain postal code and corresponding latitude, longitude
df_geo=pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge two dataframes to display postal code and corresponding Borough, Neighbourhood, latitude, longitude

In [12]:
result = pd.merge(df,df_geo,on="Postal Code")
result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
